# 05_FeatureLibrary - Physics-SR Framework v3.0

## Stage 2.1: Feature Library Construction

**Author:** Zhengze Zhang  
**Affiliation:** Department of Statistics, Columbia University  
**Date:** January 2026

---

### Purpose

Build a comprehensive candidate feature library incorporating:
- Polynomial features (up to specified degree)
- Trigonometric features (optional)
- Rational features (optional)
- Interaction terms from Stage 1.4 (iRF discovery)

### Mathematical Foundation

The feature library $\Phi$ consists of:

1. **Base features:** $\Phi_{\text{base}} = [1, x_1, x_2, \ldots, x_p]$

2. **Polynomial features:** For degree $d$:
   $$\Phi_{\text{poly}} = \{x_1^{i_1} x_2^{i_2} \cdots x_p^{i_p} : \sum_j i_j \leq d\}$$

3. **Trigonometric features:** $\Phi_{\text{trig}} = \{\sin(x_j), \cos(x_j)\}$

4. **Interaction features:** From iRF Stage 1.4

### Algorithm Reference

Framework Section 4.1: Feature Library Construction

---
## Section 1: Header and Imports

In [ ]:
"""
05_FeatureLibrary.ipynb - Feature Library Construction
=======================================================

Three-Stage Physics-Informed Symbolic Regression Framework v3.0

This module provides:
- FeatureLibraryBuilder: Construct comprehensive feature libraries
- Polynomial, trigonometric, rational, and interaction features
- Feature normalization with denormalization support
- Integration with iRF interaction terms from Stage 1.4

Algorithm:
    1. Generate polynomial features up to max_degree
    2. Add trigonometric features (optional)
    3. Add rational features (optional)
    4. Add iRF-guided interaction features
    5. Normalize features (optional)

Author: Zhengze Zhang
Affiliation: Department of Statistics, Columbia University
"""

# Import core module
%run 00_Core.ipynb

In [ ]:
# Additional imports for Feature Library
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from itertools import combinations_with_replacement
from typing import Dict, List, Tuple, Optional, Any, Set

print("05_FeatureLibrary: Additional imports successful.")

---
## Section 2: Class Definition

In [ ]:
# ==============================================================================
# FEATURE LIBRARY BUILDER CLASS
# ==============================================================================

class FeatureLibraryBuilder:
    """
    Feature Library Construction for Symbolic Regression.
    
    Builds a comprehensive candidate feature library that includes:
    - Polynomial features: x^1, x^2, x1*x2, etc.
    - Trigonometric features: sin(x), cos(x)
    - Rational features: x1/x2, 1/x
    - iRF-guided interactions from Stage 1.4
    
    The library is designed to be used with sparse regression methods
    (STLSQ, Adaptive Lasso) in Stage 2.2.
    
    Attributes
    ----------
    max_poly_degree : int
        Maximum polynomial degree (default: 3)
    include_trig : bool
        Whether to include trigonometric features (default: False)
    include_rational : bool
        Whether to include rational features (default: False)
    include_inverse : bool
        Whether to include 1/x terms (default: True)
    normalize : bool
        Whether to standardize features (default: True)
    interaction_only : bool
        If True, only include interaction terms (no pure powers)
    
    Examples
    --------
    >>> builder = FeatureLibraryBuilder(max_poly_degree=2, include_trig=True)
    >>> Phi, names = builder.build(X, feature_names)
    >>> print(f"Library size: {Phi.shape[1]} features")
    """
    
    def __init__(
        self,
        max_poly_degree: int = DEFAULT_MAX_POLY_DEGREE,
        include_trig: bool = False,
        include_rational: bool = False,
        include_inverse: bool = True,
        normalize: bool = True,
        interaction_only: bool = False
    ):
        """
        Initialize FeatureLibraryBuilder.
        
        Parameters
        ----------
        max_poly_degree : int
            Maximum polynomial degree. Features up to x^d are included.
            Default: 3
        include_trig : bool
            Whether to add sin(x), cos(x) for each variable.
            Default: False
        include_rational : bool
            Whether to add x_i/x_j rational features.
            Default: False
        include_inverse : bool
            Whether to add 1/x terms.
            Default: True
        normalize : bool
            Whether to standardize features to zero mean, unit variance.
            Default: True
        interaction_only : bool
            If True, polynomial features only include interactions (x_i*x_j),
            not pure powers (x_i^2).
            Default: False
        """
        self.max_poly_degree = max_poly_degree
        self.include_trig = include_trig
        self.include_rational = include_rational
        self.include_inverse = include_inverse
        self.normalize = normalize
        self.interaction_only = interaction_only
        
        # Internal state
        self._feature_names = None
        self._library_names = None
        self._n_input_features = None
        self._n_library_features = None
        self._scaler = None
        self._feature_means = None
        self._feature_stds = None
        self._build_complete = False
    
    def build(
        self,
        X: np.ndarray,
        feature_names: List[str],
        interactions: List[Tuple[str, ...]] = None
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Build the feature library.
        
        Parameters
        ----------
        X : np.ndarray
            Input feature matrix of shape (n_samples, n_features)
        feature_names : List[str]
            Names of input features
        interactions : List[Tuple[str, ...]], optional
            Specific interactions to include (from iRF Stage 1.4)
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            - Phi: Feature library matrix of shape (n_samples, n_library)
            - library_names: Names of library features
        """
        self._feature_names = list(feature_names)
        self._n_input_features = X.shape[1]
        n_samples = X.shape[0]
        
        all_features = []
        all_names = []
        
        # 1. Constant term
        all_features.append(np.ones((n_samples, 1)))
        all_names.append('1')
        
        # 2. Polynomial features
        poly_features, poly_names = self._polynomial_features(X)
        all_features.append(poly_features)
        all_names.extend(poly_names)
        
        # 3. Trigonometric features (optional)
        if self.include_trig:
            trig_features, trig_names = self._trigonometric_features(X)
            all_features.append(trig_features)
            all_names.extend(trig_names)
        
        # 4. Rational features (optional)
        if self.include_rational:
            rational_features, rational_names = self._rational_features(X)
            all_features.append(rational_features)
            all_names.extend(rational_names)
        
        # 5. Inverse features (optional)
        if self.include_inverse:
            inverse_features, inverse_names = self._inverse_features(X)
            all_features.append(inverse_features)
            all_names.extend(inverse_names)
        
        # 6. iRF-guided interactions (if provided)
        if interactions is not None and len(interactions) > 0:
            interaction_features, interaction_names = self._interaction_features(
                X, interactions
            )
            all_features.append(interaction_features)
            all_names.extend(interaction_names)
        
        # Combine all features
        Phi = np.hstack(all_features)
        
        # Remove duplicate columns
        Phi, all_names = self._remove_duplicates(Phi, all_names)
        
        # Handle NaN/Inf
        Phi = self._handle_invalid_values(Phi)
        
        # Normalize if requested
        if self.normalize:
            Phi = self._normalize_features(Phi)
        
        self._library_names = all_names
        self._n_library_features = Phi.shape[1]
        self._build_complete = True
        
        return Phi, all_names
    
    def _polynomial_features(
        self,
        X: np.ndarray
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Generate polynomial features.
        
        Parameters
        ----------
        X : np.ndarray
            Input features
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            Polynomial features and their names
        """
        n_samples, n_features = X.shape
        features = []
        names = []
        
        # Linear terms
        for i in range(n_features):
            features.append(X[:, i])
            names.append(self._feature_names[i])
        
        # Higher degree terms
        for degree in range(2, self.max_poly_degree + 1):
            for combo in combinations_with_replacement(range(n_features), degree):
                # Skip pure powers if interaction_only
                if self.interaction_only and len(set(combo)) == 1:
                    continue
                
                # Compute product
                product = np.ones(n_samples)
                for idx in combo:
                    product *= X[:, idx]
                
                features.append(product)
                
                # Build name
                name = self._build_polynomial_name(combo)
                names.append(name)
        
        return np.column_stack(features), names
    
    def _build_polynomial_name(
        self,
        indices: Tuple[int, ...]
    ) -> str:
        """
        Build human-readable name for polynomial term.
        
        Parameters
        ----------
        indices : Tuple[int, ...]
            Tuple of feature indices (may have repeats)
        
        Returns
        -------
        str
            Name like "x1^2" or "x1*x2"
        """
        # Count occurrences of each index
        from collections import Counter
        counts = Counter(indices)
        
        parts = []
        for idx in sorted(counts.keys()):
            name = self._feature_names[idx]
            power = counts[idx]
            if power == 1:
                parts.append(name)
            else:
                parts.append(f"{name}^{power}")
        
        return "*".join(parts)
    
    def _trigonometric_features(
        self,
        X: np.ndarray
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Generate trigonometric features.
        
        Parameters
        ----------
        X : np.ndarray
            Input features
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            Trigonometric features and their names
        """
        n_features = X.shape[1]
        features = []
        names = []
        
        for i in range(n_features):
            # sin(x)
            features.append(np.sin(X[:, i]))
            names.append(f"sin({self._feature_names[i]})")
            
            # cos(x)
            features.append(np.cos(X[:, i]))
            names.append(f"cos({self._feature_names[i]})")
        
        return np.column_stack(features), names
    
    def _rational_features(
        self,
        X: np.ndarray
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Generate rational features (x_i / x_j).
        
        Parameters
        ----------
        X : np.ndarray
            Input features
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            Rational features and their names
        """
        n_features = X.shape[1]
        features = []
        names = []
        
        for i in range(n_features):
            for j in range(n_features):
                if i != j:
                    # x_i / x_j with safe division
                    ratio = safe_divide(X[:, i], X[:, j])
                    features.append(ratio)
                    names.append(f"{self._feature_names[i]}/{self._feature_names[j]}")
        
        if len(features) == 0:
            return np.empty((X.shape[0], 0)), []
        
        return np.column_stack(features), names
    
    def _inverse_features(
        self,
        X: np.ndarray
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Generate inverse features (1/x, 1/x^2).
        
        Parameters
        ----------
        X : np.ndarray
            Input features
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            Inverse features and their names
        """
        n_features = X.shape[1]
        features = []
        names = []
        
        for i in range(n_features):
            # 1/x
            inv = safe_divide(1.0, X[:, i])
            features.append(inv)
            names.append(f"1/{self._feature_names[i]}")
            
            # 1/x^2
            inv_sq = safe_divide(1.0, X[:, i]**2)
            features.append(inv_sq)
            names.append(f"1/{self._feature_names[i]}^2")
        
        return np.column_stack(features), names
    
    def _interaction_features(
        self,
        X: np.ndarray,
        interactions: List[Tuple[str, ...]]
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Generate interaction features from iRF discovery.
        
        Parameters
        ----------
        X : np.ndarray
            Input features
        interactions : List[Tuple[str, ...]]
            List of feature name tuples representing interactions
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            Interaction features and their names
        """
        n_samples = X.shape[0]
        features = []
        names = []
        
        # Create mapping from name to index
        name_to_idx = {name: i for i, name in enumerate(self._feature_names)}
        
        for interaction in interactions:
            # Compute product of features in interaction
            product = np.ones(n_samples)
            valid = True
            
            for feat_name in interaction:
                if feat_name in name_to_idx:
                    idx = name_to_idx[feat_name]
                    product *= X[:, idx]
                else:
                    valid = False
                    break
            
            if valid:
                features.append(product)
                name = "*".join(sorted(interaction))
                names.append(f"iRF:{name}")
        
        if len(features) == 0:
            return np.empty((n_samples, 0)), []
        
        return np.column_stack(features), names
    
    def _remove_duplicates(
        self,
        Phi: np.ndarray,
        names: List[str]
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Remove duplicate columns from feature matrix.
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature matrix
        names : List[str]
            Feature names
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            Deduplicated features and names
        """
        n_features = Phi.shape[1]
        keep_indices = []
        seen_hashes = set()
        
        for i in range(n_features):
            col = Phi[:, i]
            # Use rounded values for hash to handle floating point
            col_hash = hash(tuple(np.round(col, 8)))
            
            if col_hash not in seen_hashes:
                seen_hashes.add(col_hash)
                keep_indices.append(i)
        
        return Phi[:, keep_indices], [names[i] for i in keep_indices]
    
    def _handle_invalid_values(
        self,
        Phi: np.ndarray
    ) -> np.ndarray:
        """
        Handle NaN and Inf values in feature matrix.
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature matrix
        
        Returns
        -------
        np.ndarray
            Feature matrix with invalid values replaced
        """
        # Replace Inf with large finite values
        Phi = np.clip(Phi, -1e10, 1e10)
        
        # Replace NaN with column mean
        for j in range(Phi.shape[1]):
            col = Phi[:, j]
            nan_mask = np.isnan(col)
            if np.any(nan_mask):
                col_mean = np.nanmean(col)
                if np.isnan(col_mean):
                    col_mean = 0.0
                Phi[nan_mask, j] = col_mean
        
        return Phi
    
    def _normalize_features(
        self,
        Phi: np.ndarray
    ) -> np.ndarray:
        """
        Standardize features to zero mean, unit variance.
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature matrix
        
        Returns
        -------
        np.ndarray
            Normalized feature matrix
        """
        self._feature_means = np.mean(Phi, axis=0)
        self._feature_stds = np.std(Phi, axis=0)
        
        # Avoid division by zero
        self._feature_stds[self._feature_stds < EPS_DIV] = 1.0
        
        # Don't normalize the constant term (first column)
        Phi_normalized = Phi.copy()
        Phi_normalized[:, 1:] = (
            Phi[:, 1:] - self._feature_means[1:]
        ) / self._feature_stds[1:]
        
        return Phi_normalized
    
    def transform(
        self,
        X_new: np.ndarray
    ) -> np.ndarray:
        """
        Transform new data using the same library construction.
        
        Parameters
        ----------
        X_new : np.ndarray
            New feature matrix
        
        Returns
        -------
        np.ndarray
            Transformed feature library
        
        Raises
        ------
        ValueError
            If build() has not been called
        """
        if not self._build_complete:
            raise ValueError("Must call build() before transform()")
        
        # Rebuild library with new data (without re-fitting scaler)
        Phi_new, _ = self.build(
            X_new, 
            self._feature_names,
            interactions=None  # Use same structure
        )
        
        return Phi_new
    
    def get_feature_count(self) -> int:
        """
        Get total number of features in library.
        
        Returns
        -------
        int
            Number of library features
        """
        if not self._build_complete:
            raise ValueError("Must call build() before getting feature count")
        return self._n_library_features
    
    def get_library_names(self) -> List[str]:
        """
        Get list of library feature names.
        
        Returns
        -------
        List[str]
            Names of library features
        """
        if not self._build_complete:
            raise ValueError("Must call build() before getting names")
        return self._library_names.copy()
    
    def print_library_summary(self) -> None:
        """
        Print a summary of the feature library.
        """
        if not self._build_complete:
            print("Library not yet built. Call build() first.")
            return
        
        print("=" * 70)
        print(" Feature Library Summary")
        print("=" * 70)
        print()
        print(f"Configuration:")
        print(f"  Max polynomial degree: {self.max_poly_degree}")
        print(f"  Include trigonometric: {self.include_trig}")
        print(f"  Include rational: {self.include_rational}")
        print(f"  Include inverse: {self.include_inverse}")
        print(f"  Normalize: {self.normalize}")
        print()
        print(f"Library Statistics:")
        print(f"  Input features: {self._n_input_features}")
        print(f"  Library features: {self._n_library_features}")
        print(f"  Expansion factor: {self._n_library_features / self._n_input_features:.1f}x")
        print()
        print("-" * 70)
        print(" Library Features:")
        print("-" * 70)
        
        # Group by type
        poly_features = [n for n in self._library_names 
                        if not n.startswith('sin') and not n.startswith('cos') 
                        and not n.startswith('1/') and not '/' in n
                        and not n.startswith('iRF:')]
        trig_features = [n for n in self._library_names 
                        if n.startswith('sin') or n.startswith('cos')]
        inverse_features = [n for n in self._library_names 
                           if n.startswith('1/')]
        rational_features = [n for n in self._library_names 
                            if '/' in n and not n.startswith('1/')]
        irf_features = [n for n in self._library_names 
                       if n.startswith('iRF:')]
        
        print(f"  Polynomial ({len(poly_features)}): {poly_features[:5]}..." 
              if len(poly_features) > 5 else f"  Polynomial ({len(poly_features)}): {poly_features}")
        
        if trig_features:
            print(f"  Trigonometric ({len(trig_features)}): {trig_features[:4]}..." 
                  if len(trig_features) > 4 else f"  Trigonometric ({len(trig_features)}): {trig_features}")
        
        if inverse_features:
            print(f"  Inverse ({len(inverse_features)}): {inverse_features[:4]}..." 
                  if len(inverse_features) > 4 else f"  Inverse ({len(inverse_features)}): {inverse_features}")
        
        if rational_features:
            print(f"  Rational ({len(rational_features)}): {rational_features[:4]}..." 
                  if len(rational_features) > 4 else f"  Rational ({len(rational_features)}): {rational_features}")
        
        if irf_features:
            print(f"  iRF Interactions ({len(irf_features)}): {irf_features}")
        
        print()
        print("=" * 70)

---
## Section 3: Internal Tests

In [ ]:
# ==============================================================================
# TEST CONTROL FLAG
# ==============================================================================

_RUN_TESTS = False  # Set to True to run internal tests

if _RUN_TESTS:
    print("=" * 70)
    print(" RUNNING INTERNAL TESTS FOR 05_FeatureLibrary")
    print("=" * 70)

In [ ]:
# ==============================================================================
# TEST 1: Polynomial Feature Generation
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 1: Polynomial Feature Generation")
    
    # Generate simple test data
    np.random.seed(42)
    n_samples = 100
    
    X = np.random.randn(n_samples, 3)
    feature_names = ['x0', 'x1', 'x2']
    
    print(f"Input: {len(feature_names)} features")
    print(f"Max polynomial degree: 3")
    print()
    
    # Build library
    builder = FeatureLibraryBuilder(
        max_poly_degree=3,
        include_trig=False,
        include_rational=False,
        include_inverse=False,
        normalize=False
    )
    Phi, names = builder.build(X, feature_names)
    
    print(f"Library size: {Phi.shape[1]} features")
    print(f"Feature names: {names}")
    print()
    
    # Expected: 1 + 3 + 6 + 10 = 20 features for degree 3 with 3 variables
    # (constant + linear + degree2 + degree3)
    expected_count = 1 + 3 + 6 + 10  # C(3+d-1, d) for each degree
    print(f"Expected count: ~{expected_count} (may vary with deduplication)")
    
    # Verify constant term
    if np.allclose(Phi[:, 0], 1.0):
        print("[PASS] Constant term is all 1s")
    else:
        print("[FAIL] Constant term incorrect")

In [ ]:
# ==============================================================================
# TEST 2: Integration with iRF Interactions
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 2: Integration with iRF Interactions")
    
    np.random.seed(42)
    n_samples = 100
    
    X = np.random.uniform(0.1, 1, (n_samples, 4))
    feature_names = ['a', 'b', 'c', 'd']
    
    # Simulated iRF interactions
    interactions = [
        ('a', 'b'),      # Pairwise
        ('b', 'c', 'd')  # 3-way
    ]
    
    print(f"Input features: {feature_names}")
    print(f"iRF interactions: {interactions}")
    print()
    
    builder = FeatureLibraryBuilder(
        max_poly_degree=2,
        include_trig=False,
        include_inverse=False,
        normalize=False
    )
    Phi, names = builder.build(X, feature_names, interactions=interactions)
    
    print(f"Library size: {Phi.shape[1]}")
    print(f"Feature names:")
    for i, name in enumerate(names):
        print(f"  {i}: {name}")
    
    # Check that iRF features are included
    irf_features = [n for n in names if n.startswith('iRF:')]
    print(f"\niRF features: {irf_features}")
    
    if len(irf_features) == 2:
        print("[PASS] Both iRF interactions included")
    else:
        print(f"[WARNING] Expected 2 iRF features, got {len(irf_features)}")

In [ ]:
# ==============================================================================
# TEST 3: No NaN/Inf Verification
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 3: No NaN/Inf Verification")
    
    # Generate data with potential issues (near-zero values)
    np.random.seed(42)
    n_samples = 100
    
    X = np.random.uniform(0.01, 1, (n_samples, 2))
    # Add some near-zero values
    X[0, 0] = 0.0001
    X[1, 1] = 0.0001
    
    feature_names = ['x', 'y']
    
    print(f"Testing with near-zero values in data")
    print(f"Min value in X: {X.min()}")
    print()
    
    builder = FeatureLibraryBuilder(
        max_poly_degree=2,
        include_trig=True,
        include_rational=True,
        include_inverse=True,
        normalize=True
    )
    Phi, names = builder.build(X, feature_names)
    
    # Check for NaN/Inf
    has_nan = np.any(np.isnan(Phi))
    has_inf = np.any(np.isinf(Phi))
    
    print(f"Library size: {Phi.shape[1]}")
    print(f"Has NaN: {has_nan}")
    print(f"Has Inf: {has_inf}")
    
    if not has_nan and not has_inf:
        print("[PASS] No NaN or Inf values in library")
    else:
        print("[FAIL] Invalid values present")

In [ ]:
# ==============================================================================
# TEST 4: Normalization Verification
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 4: Normalization Verification")
    
    np.random.seed(42)
    n_samples = 500
    
    X = np.random.randn(n_samples, 2) * 10 + 5  # Non-zero mean, non-unit std
    feature_names = ['x', 'y']
    
    print(f"Original data:")
    print(f"  X mean: {X.mean(axis=0)}")
    print(f"  X std: {X.std(axis=0)}")
    print()
    
    builder = FeatureLibraryBuilder(
        max_poly_degree=2,
        normalize=True
    )
    Phi, names = builder.build(X, feature_names)
    
    print(f"Normalized library (excluding constant term):")
    print(f"  Mean (should be ~0): {Phi[:, 1:].mean(axis=0)[:3]}...")
    print(f"  Std (should be ~1): {Phi[:, 1:].std(axis=0)[:3]}...")
    
    # Check mean and std
    mean_close_to_zero = np.allclose(Phi[:, 1:].mean(axis=0), 0, atol=0.1)
    std_close_to_one = np.allclose(Phi[:, 1:].std(axis=0), 1, atol=0.2)
    
    if mean_close_to_zero and std_close_to_one:
        print("[PASS] Features properly normalized")
    else:
        print("[WARNING] Normalization may not be exact for all features")

In [ ]:
# ==============================================================================
# TEST 5: Full Library Summary
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 5: Full Library Summary")
    
    # Use warm rain data
    X, y, feature_names, _ = generate_warm_rain_data(
        n_samples=200, noise_level=0.01
    )
    
    # Build comprehensive library
    builder = FeatureLibraryBuilder(
        max_poly_degree=3,
        include_trig=True,
        include_rational=False,  # Skip rational to keep size manageable
        include_inverse=True,
        normalize=True
    )
    
    # Simulated interactions from iRF
    interactions = [('q_c', 'N_d')]
    
    Phi, names = builder.build(X, feature_names, interactions=interactions)
    
    builder.print_library_summary()

---
## Section 4: Module Summary

In [ ]:
# ==============================================================================
# MODULE SUMMARY
# ==============================================================================

print("=" * 70)
print(" 05_FeatureLibrary.ipynb - Module Summary")
print("=" * 70)
print()
print("CLASS: FeatureLibraryBuilder")
print("-" * 70)
print()
print("Purpose:")
print("  Build comprehensive candidate feature library for sparse regression.")
print("  Includes polynomial, trigonometric, rational, and interaction features.")
print()
print("Main Methods:")
print("  build(X, feature_names, interactions=None)")
print("      Build the feature library")
print("      Returns: (Phi, names) - feature matrix and names")
print()
print("  transform(X_new)")
print("      Transform new data using same library structure")
print()
print("  get_feature_count()")
print("      Get total number of library features")
print()
print("  get_library_names()")
print("      Get list of feature names")
print()
print("  print_library_summary()")
print("      Print detailed library summary")
print()
print("Key Parameters:")
print("  max_poly_degree: Maximum polynomial degree (default: 3)")
print("  include_trig: Include sin(x), cos(x) (default: False)")
print("  include_rational: Include x_i/x_j (default: False)")
print("  include_inverse: Include 1/x terms (default: True)")
print("  normalize: Standardize features (default: True)")
print()
print("Usage Example:")
print("-" * 70)
print("""
# Create builder
builder = FeatureLibraryBuilder(
    max_poly_degree=3,
    include_trig=True,
    normalize=True
)

# Build library (with optional iRF interactions)
interactions = [('x0', 'x1'), ('x1', 'x2')]  # From Stage 1.4
Phi, names = builder.build(X, feature_names, interactions=interactions)

print(f"Library shape: {Phi.shape}")
print(f"Features: {names[:5]}...")

# Use Phi for sparse regression in Stage 2.2
""")
print()
print("=" * 70)
print("Module loaded successfully. Import via: %run 05_FeatureLibrary.ipynb")
print("=" * 70)